<a href="https://www.kaggle.com/code/stpeteishii/fifa20-create-database-w-crud?scriptVersionId=218782377" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# FIFA20 Create Database w/CRUD

In [1]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/fifa-20-complete-player-dataset/players_15.csv
/kaggle/input/fifa-20-complete-player-dataset/players_16.csv
/kaggle/input/fifa-20-complete-player-dataset/players_17.csv
/kaggle/input/fifa-20-complete-player-dataset/players_18.csv
/kaggle/input/fifa-20-complete-player-dataset/players_19.csv
/kaggle/input/fifa-20-complete-player-dataset/teams_and_leagues.csv
/kaggle/input/fifa-20-complete-player-dataset/players_20.csv


# Prepare DataFrame from CSV

In [2]:
player0=pd.read_csv('../input/fifa-20-complete-player-dataset/players_20.csv')
player0[0:2].T

,0,1
sofifa_id,158023,20801
player_url,https://sofifa.com/player/158023/lionel-messi/...,https://sofifa.com/player/20801/c-ronaldo-dos-...
short_name,L. Messi,Cristiano Ronaldo
long_name,Lionel Andrés Messi Cuccittini,Cristiano Ronaldo dos Santos Aveiro
age,32,34
...,...,...
lb,63+2,61+3
lcb,52+2,53+3
cb,52+2,53+3
rcb,52+2,53+3


In [3]:
# L. Messi
print(player0.iloc[0,1])
print()
player0[player0['short_name']=='L. Messi']

https://sofifa.com/player/158023/lionel-messi/20/159586



,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,32,1987-06-24,170,72,Argentina,FC Barcelona,...,68+2,66+2,66+2,66+2,68+2,63+2,52+2,52+2,52+2,63+2


![https://dynaimage.cdn.cnn.com/cnn/c_fill,g_auto,w_1200,h_675,ar_16:9/https%3A%2F%2Fcdn.cnn.com%2Fcnnnext%2Fdam%2Fassets%2F210910103403-lionel-messi-argentina-bolivia.jpg](https://dynaimage.cdn.cnn.com/cnn/c_fill,g_auto,w_1200,h_675,ar_16:9/https%3A%2F%2Fcdn.cnn.com%2Fcnnnext%2Fdam%2Fassets%2F210910103403-lionel-messi-argentina-bolivia.jpg)

In [4]:
print(len(player0))
print(player0.columns.tolist())

18278
['sofifa_id', 'player_url', 'short_name', 'long_name', 'age', 'dob', 'height_cm', 'weight_kg', 'nationality', 'club', 'overall', 'potential', 'value_eur', 'wage_eur', 'player_positions', 'preferred_foot', 'international_reputation', 'weak_foot', 'skill_moves', 'work_rate', 'body_type', 'real_face', 'release_clause_eur', 'player_tags', 'team_position', 'team_jersey_number', 'loaned_from', 'joined', 'contract_valid_until', 'nation_position', 'nation_jersey_number', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'gk_diving', 'gk_handling', 'gk_kicking', 'gk_reflexes', 'gk_speed', 'gk_positioning', 'player_traits', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power', '

In [5]:
player=player0[0:100]
print(len(player))

100


# Import SQLAlchemy to Create Database
https://github.com/sqlalchemy/sqlalchemy

In [6]:
!pip3 install sqlalchemy

In [7]:
from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column,Integer,Float,String
from sqlalchemy import desc, asc

In [8]:
#!rm data.db

In [9]:
database_file=os.path.join(os.path.abspath(os.getcwd()),'data.db')
database_file

'/kaggle/working/data.db'

In [10]:
engine=create_engine('sqlite:///'+database_file,convert_unicode=True,echo=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SADeprecationWarning: The create_engine.convert_unicode parameter and corresponding dialect-level parameters are deprecated, and will be removed in a future release.  Modern DBAPIs support Python Unicode natively and this parameter is unnecessary.
  """Entry point for launching an IPython kernel.


In [11]:
db_session=scoped_session(
    sessionmaker(
        autocommit=False,
        autoflush=False,
        bind=engine
))

In [12]:
Base=declarative_base()
Base.query=db_session.query_property()

In [13]:
class FIFA(Base):
    __tablename__='player'
    id=Column(Integer,primary_key=True)
    short_name=Column(String(255),unique=False)
    age=Column(Integer,unique=False)
    nationality=Column(String(255),unique=False)
    club=Column(String(255),unique=False)
    
    def __int__(self,id=None,long_name=None,age=None,nationality=None,club=None):
        self.id=id
        self.short_name=long_name
        self.age=age
        self.nationality=nationality
        self.club=club

In [14]:
Base.metadata.create_all(bind=engine)

2025-01-22 14:36:03,690 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-22 14:36:03,692 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("player")
2025-01-22 14:36:03,693 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-22 14:36:03,695 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("player")
2025-01-22 14:36:03,696 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-22 14:36:03,699 INFO sqlalchemy.engine.Engine 
CREATE TABLE player (
	id INTEGER NOT NULL, 
	short_name VARCHAR(255), 
	age INTEGER, 
	nationality VARCHAR(255), 
	club VARCHAR(255), 
	PRIMARY KEY (id)
)


2025-01-22 14:36:03,700 INFO sqlalchemy.engine.Engine [no key 0.00111s] ()
2025-01-22 14:36:03,717 INFO sqlalchemy.engine.Engine COMMIT


In [15]:
def read_data():

    for index,_df in player.iterrows():
        row=FIFA(short_name=_df['short_name'],age=_df['age'],nationality=_df['nationality'],club=_df['club'])
        db_session.add(row)
        
    db_session.commit()

In [16]:
read_data()

2025-01-22 14:36:03,842 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-22 14:36:03,847 INFO sqlalchemy.engine.Engine INSERT INTO player (short_name, age, nationality, club) VALUES (?, ?, ?, ?)
2025-01-22 14:36:03,848 INFO sqlalchemy.engine.Engine [generated in 0.00151s] ('L. Messi', 32, 'Argentina', 'FC Barcelona')
2025-01-22 14:36:03,850 INFO sqlalchemy.engine.Engine INSERT INTO player (short_name, age, nationality, club) VALUES (?, ?, ?, ?)
2025-01-22 14:36:03,851 INFO sqlalchemy.engine.Engine [cached since 0.004279s ago] ('Cristiano Ronaldo', 34, 'Portugal', 'Juventus')
2025-01-22 14:36:03,853 INFO sqlalchemy.engine.Engine INSERT INTO player (short_name, age, nationality, club) VALUES (?, ?, ?, ?)
2025-01-22 14:36:03,854 INFO sqlalchemy.engine.Engine [cached since 0.006791s ago] ('Neymar Jr', 27, 'Brazil', 'Paris Saint-Germain')
2025-01-22 14:36:03,855 INFO sqlalchemy.engine.Engine INSERT INTO player (short_name, age, nationality, club) VALUES (?, ?, ?, ?)
2025-01-22 14:36:0

# CRUD operation sample
CRUD: create, read, update and delete

In [17]:
db=db_session.query(FIFA).filter(FIFA.short_name=='L. Messi').all()
print()
for row in db:
    print(row.id,row.short_name,row.age)

2025-01-22 14:36:04,410 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-22 14:36:04,414 INFO sqlalchemy.engine.Engine SELECT player.id AS player_id, player.short_name AS player_short_name, player.age AS player_age, player.nationality AS player_nationality, player.club AS player_club 
FROM player 
WHERE player.short_name = ?
2025-01-22 14:36:04,415 INFO sqlalchemy.engine.Engine [generated in 0.00153s] ('L. Messi',)

1 L. Messi 32


In [18]:
db=db_session.query(FIFA).order_by(desc(FIFA.age)).all()
print()
for i,row in enumerate(db):
    if i<10:
        print(row.id,row.short_name,row.age)

2025-01-22 14:36:04,572 INFO sqlalchemy.engine.Engine SELECT player.id AS player_id, player.short_name AS player_short_name, player.age AS player_age, player.nationality AS player_nationality, player.club AS player_club 
FROM player ORDER BY player.age DESC
2025-01-22 14:36:04,574 INFO sqlalchemy.engine.Engine [generated in 0.00214s] ()

97 Z. Ibrahimović 37
2 Cristiano Ronaldo 34
17 G. Chiellini 34
31 S. Handanovič 34
45 Fernandinho 34
46 Thiago Silva 34
9 L. Modrić 33
19 Sergio Ramos 33
32 M. Neuer 33
34 David Silva 33


# Import Sqlite3 to Convert Database to DataFrame

In [19]:
import sqlite3

In [20]:
conn = sqlite3.connect(r'./data.db')
conn

In [21]:
query = 'SELECT SHORT_NAME,AGE,NATIONALITY,CLUB FROM PLAYER'
db0 = pd.read_sql_query(query,conn)
db0

,short_name,age,nationality,club
0,L. Messi,32,Argentina,FC Barcelona
1,Cristiano Ronaldo,34,Portugal,Juventus
2,Neymar Jr,27,Brazil,Paris Saint-Germain
3,J. Oblak,26,Slovenia,Atlético Madrid
4,E. Hazard,28,Belgium,Real Madrid
...,...,...,...,...
95,H. Ziyech,26,Morocco,Ajax
96,Z. Ibrahimović,37,Sweden,LA Galaxy
97,A. Gómez,31,Argentina,Atalanta
98,G. Higuaín,31,Argentina,Juventus


In [22]:
db0[db0['short_name']=='L. Messi']

,short_name,age,nationality,club
0,L. Messi,32,Argentina,FC Barcelona


In [23]:
#!rm data.db